<a href="https://colab.research.google.com/github/davidpilecek/Pramakon/blob/master/Pramakon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq fastai duckduckgo_search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.1.3 which is incompatible.


In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *
from time import sleep
import csv
import os
from google.colab import drive
import csv

warnings.filterwarnings("ignore", category=UserWarning) 

species = 'salvej', 'kopriva'
path_to_predict = r'/content/gdrive/MyDrive/Colab_Notebooks/unclassified_pics/'
path = Path('/content/gdrive/MyDrive/Colab_Notebooks/dataset')
path_model = Path('/content/gdrive/MyDrive/Colab_Notebooks/fastai_model.pkl')

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Colab_Notebooks/unclassified_pics

/content/gdrive/MyDrive/Colab_Notebooks/unclassified_pics


In [ ]:
def classify_dir(path_to_predict = path_to_predict):
    _, _, files = next(os.walk(path_to_predict))
    num_of_pics = len(files)
    i = 0
    for i in range(num_of_pics):
        path = f"{path_to_predict}/img{i}.jpg"
        print(path)

In [ ]:
def search_images(term, max_images=200):
    print(f"Searching for '{term}'")
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
def create_dataset():
    shutil.rmtree(path)
    for o in species:
        dest = (path/o)
        dest.mkdir(exist_ok=True, parents=True)
        download_images(dest, urls=search_images(f'diseased {o} photo'))
        sleep(10)
        download_images(dest, urls=search_images(f'{o} photo'))
        sleep(10)
        download_images(dest, urls=search_images(f'{o} sun photo'))
        sleep(10)
        download_images(dest, urls=search_images(f'{o} shade photo'))
        sleep(10)
        resize_images(path/o, max_size=400, dest=path/o)

    failed = verify_images(get_image_files(path))
    failed.map(Path.unlink)

In [ ]:
def learn():
    dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
    ).dataloaders(path, bs=32)

    learn = vision_learner(dls, resnet50, metrics=error_rate)
    learn.fine_tune(8)

    learn.export(path_model)

In [ ]:
def predict(path_to_predict = path_to_predict):
    results = []
    learn = load_learner(path_model)
    _, _, files = next(os.walk(path_to_predict))
    num_of_pics = len(files)
    i = 1
    for i in range(num_of_pics):
        path_img = f"{path_to_predict}/img{i}.jpg"
        print(path_img)
        class_obj,_,probs = learn.predict(PILImage.create(path_img))
        print(f"This is a: {class_obj}.")
        print(f"Probability it's a {class_obj}: {max(probs):.4f}")
        results.append(class_obj)
        results.append(f"{max(probs):.4f}")
    return results

In [ ]:
def create_csv(results):
  num_of_cells = int(len(results))
  print(num_of_cells)
  with open('/content/results.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Rostlinka", "Pravdepodobnost"])
        for i in range(0,num_of_cells-1 , 2):
            writer.writerow([f"{results[i]}",f"{results[i+1]}"])
  csvfile.close()      





In [ ]:
if __name__ == '__main__':
    results = predict()
    print(results)
    create_csv(results)

/content/gdrive/MyDrive/Colab_Notebooks/unclassified_pics//img0.jpg


This is a: unclassified_pics.
Probability it's a unclassified_pics: 0.9894
/content/gdrive/MyDrive/Colab_Notebooks/unclassified_pics//img1.jpg


This is a: sage leaf.
Probability it's a sage leaf: 0.9558
/content/gdrive/MyDrive/Colab_Notebooks/unclassified_pics//img2.jpg


This is a: nettle leaf.
Probability it's a nettle leaf: 0.9936
['unclassified_pics', '0.9894', 'sage leaf', '0.9558', 'nettle leaf', '0.9936']
6
